In [1]:
from Bio import SeqIO
from collections import Counter
import pandas as pd 
import re
import requests
import json
import time
import glob
import pprint
import seaborn as sns
import pylab as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import pycountry 
import pycountry_convert as pc
pp = pprint.PrettyPrinter()
import numpy as np
from tabulate import tabulate
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import regex as re
import plotly.graph_objects as go
import os

from string import punctuation
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
stemmer=SnowballStemmer("english")
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import math

import networkx as nx
#from fa2 import ForceAtlas2
#import community as community_louvain  # For Louvain method

from collections import defaultdict
from itertools import combinations


%env folder = X:\5_Research\Paul\dsi_origins\

env: folder=X:\5_Research\Paul\dsi_origins\


In [2]:
path = %env folder

In [15]:
df_communities_full = pd.read_csv(path+R'processed_data\co_word_network_+2tfidf_backbone_90_leiden_innerweighdeg_small_best.csv', index_col = 0)#.dropna()

In [17]:
df_communities_full

,community_small_res,weighted_degree,weighted_degree_within_small_com,community_medium_res,weighted_degree_within_medium_com
despite,0,356.0,175.0,6,160.0
selection,2,1727.0,720.0,6,625.0
balancing,2,19.0,13.0,1,8.0
mating,2,133.0,72.0,1,52.0
balance,2,160.0,69.0,6,59.0
...,...,...,...,...,...
albicilla,2,2.0,2.0,1,2.0
winogradskyella,2,2.0,2.0,8,2.0
wuxiang,0,2.0,2.0,6,2.0
cider,0,4.0,2.0,6,1.0


In [19]:
Counter(df_communities_full['community_small_res']).most_common()

[(2, 9040), (1, 8036), (0, 6921)]

In [21]:
#drop the one tiny community
top_communities = [0,1,2]
df_communities = df_communities_full[df_communities_full['community_small_res'].isin(top_communities)].reset_index().rename(columns = {'community_small_res':'community','index': 'word'})

In [23]:
df_communities.sort_values('weighted_degree', ascending = False)

,word,community,weighted_degree,weighted_degree_within_small_com,community_medium_res,weighted_degree_within_medium_com
574,hiv1,0,4484.0,3754.0,6,3664.0
612,influenza,0,4432.0,3801.0,6,3463.0
759,hepatitis,0,3978.0,3239.0,6,2859.0
39,distribution,2,3803.0,1625.0,1,1234.0
743,effect,1,3673.0,2130.0,0,1929.0
...,...,...,...,...,...,...
18167,bathyergidae,2,1.0,1.0,1,1.0
23886,shimia,1,1.0,1.0,6,1.0
16891,ultraconserved,0,1.0,1.0,6,1.0
22593,breakup,2,1.0,1.0,1,1.0


In [31]:
df_communities.to_csv(path+R'processed_data/leiden_word.csv')

### now find most semantically related publications for each community

In [11]:
with open(path+R'processed_data\word_id_dict_+2_tfidf.json') as json_string:
    word_id_dict = json.load(json_string)

In [13]:
#frist create the pub - word - community linkages
word_list = list(df_communities['word'].dropna())

pubs = []
communities = []
words = []

for word in word_list:
    community = list(df_communities['community'][df_communities['word'] == word])[0]
    for pub_id in word_id_dict[word]:
        pubs.append(pub_id)
        communities.append(community)
        words.append(word)
    

In [43]:
#overall how many will we not be able to link back (due to 1) words that we cut in the pre-processing and 2) the backboning)
94772 - len(set(pubs))

174

In [45]:
#how many are remaining? 
len(set(pubs))

94598

In [49]:
df_sem = pd.DataFrame(zip(pubs, communities, words)).reset_index().rename(columns = {1:'community', 0: 'id_select', 2:'word'})

grouped = df_sem.groupby(['id_select','community']).size().reset_index(name='count')
semantic_link = grouped.loc[grouped.groupby('id_select')['count'].idxmax()]


In [51]:
# Group by 'id_select' and aggregate the words into a single string
grouped = df_sem.groupby('id_select')['word'].apply(lambda words: ', '.join(words))

# Convert the result to a dictionary
words_sel_dict = grouped.to_dict()

In [53]:
semantic_link['words_sel'] = semantic_link['id_select'].map(words_sel_dict)

In [70]:
#get the original titles with the id selects
df_scholarly = pd.read_csv(path+R'processed_data\scholarly_aff_group.csv', index_col = 0)

In [71]:
df_id_title = df_scholarly[['id_select', 'title']].drop_duplicates()

In [72]:
df_id_title.to_csv(path+R'processed_data\id_title_dict.csv')

In [80]:
#merge original title with id_select
community_analysis = pd.merge(semantic_link,df_id_title, how = 'left')

In [86]:
community_analysis.to_csv(path+R'processed_data\community_analysis_3.csv', index = False)

In [25]:
# add the communities to the regression dataframe
community_analysis = pd.read_csv(path+R'processed_data\community_analysis_3.csv')
df_extract = pd.read_csv(path+R'processed_data\extractivism.csv', index_col = 0)

In [29]:
df_extract_com = pd.merge(df_extract, community_analysis[['id_select','community']], how = 'left')

In [31]:
df_extract_com_nonans = df_extract_com[df_extract_com['community'].notna()]

In [33]:
df_extract_com_nonans.to_csv(path+R'processed_data\extractivism_com3.csv', index = False)

In [35]:
#how many publications remain wiht known communities? 
len(df_extract_com_nonans)

94598

In [37]:
#how many are linked to public health
len(df_extract_com_nonans[df_extract_com_nonans['community'] == 0])

39217

In [39]:
#public health share
len(df_extract_com_nonans[df_extract_com_nonans['community'] == 0])/94598

0.41456478995327595

In [41]:
len(df_extract_com_nonans[df_extract_com_nonans['community'] == 1])

28433

In [43]:
#microbiology share
len(df_extract_com_nonans[df_extract_com_nonans['community'] == 1])/94598

0.3005666081735343

In [45]:
len(df_extract_com_nonans[df_extract_com_nonans['community'] == 2])

26948

In [47]:
#ev bio share
26948/94598

0.2848686018731897

### create the content analysis excel 

In [25]:
#reduce to top 50 words per community
top_com_words = []
for com in top_communities: 
    top_com_words.extend(list(df_communities[df_communities['community'] == com].sort_values('weighted_degree_within_small_com', ascending = False)['word'].values)[:50])

In [27]:
df_communities_topwords = df_communities[df_communities['word'].isin(top_com_words)]

In [29]:
df_communities_topwords[df_communities_topwords['community'] == 0].sort_values('weighted_degree_within_small_com', ascending = False)

,word,community,weighted_degree,weighted_degree_within_small_com,community_medium_res,weighted_degree_within_medium_com
612,influenza,0,4432.0,3801.0,6,3463.0
574,hiv1,0,4484.0,3754.0,6,3664.0
759,hepatitis,0,3978.0,3239.0,6,2859.0
886,respiratory,0,3031.0,2764.0,6,2501.0
1035,viral,0,3564.0,2587.0,6,2236.0
303,porcine,0,2961.0,2586.0,6,2280.0
617,outbreak,0,2930.0,2427.0,6,1803.0
205,syndrome,0,2600.0,2170.0,6,1897.0
1614,hiv,0,2586.0,2160.0,6,2076.0
547,case,0,3148.0,2051.0,6,1519.0


In [77]:
df_communities_topwords.to_csv(path+R'processed_data/leiden_word_analysis.csv')

In [59]:
#re-create the pub - word - community linkages for top words
word_list_topwords = list(df_communities_topwords['word'].dropna())

pubs_topwords = []
communities_topwords = []
words_topwords = []

for word in word_list_topwords:
    community = list(df_communities_topwords['community'][df_communities_topwords['word'] == word])[0]
    for pub_id in word_id_dict[word]:
        pubs_topwords.append(pub_id)
        communities_topwords.append(community)
        words_topwords.append(word)
    

In [61]:
df_sem_topwords = pd.DataFrame(zip(pubs_topwords, communities_topwords, words_topwords)).reset_index().rename(columns = {1:'community', 0: 'id_select', 2:'word'})

grouped_topwords = df_sem_topwords.groupby(['id_select','community']).size().reset_index(name='count')
semantic_link_topwords = grouped_topwords.loc[grouped_topwords.groupby('id_select')['count'].idxmax()]


In [63]:
# Group by 'id_select' and aggregate the words into a single string
grouped_topwords = df_sem_topwords.groupby('id_select')['word'].apply(lambda words: ', '.join(words))

# Convert the result to a dictionary
words_sel_dict_topwords = grouped_topwords.to_dict()

In [65]:
semantic_link_topwords['words_sel'] = semantic_link_topwords['id_select'].map(words_sel_dict_topwords)

In [69]:
#merge original title with id_select
community_analysis_topwords = pd.merge(semantic_link_topwords,df_id_title, how = 'left')

In [73]:
community_analysis_topwords.sort_values('count')

,id_select,community,count,words_sel,title
0,10.1002/jpln.201100382,1,1,involve,Characterization of new soil thermophilic bact...
35971,27697483.0,2,1,diptera,First report of Aedes (Stegomyia) albopictus (...
35972,27697556.0,0,1,"multiple, teleostei",Multiple colonisations of the Lake Malawi catc...
35973,27698394.0,1,1,role,The pioneering role of PRDM9 indel mutations i...
35974,27698581.0,0,1,province,"Lithocarpus dahuoaiensis (Fagaceae), a new spe..."
...,...,...,...,...,...
57311,pmc5625113,1,7,"salt, stress, endophytic, growth, fusarium, ro...","Endophytic Bacteria Improve Plant Growth, Symb..."
7285,16817772.0,0,7,"syndrome, porcine, response, pig, reproductive...",Neutralizing antibody responses of pigs infect...
23820,23405309.0,0,7,"syndrome, porcine, pig, reproductive, respirat...",Complete Genome Sequence of a Novel Natural Re...
52977,pmc3650200,0,7,"syndrome, porcine, highly, reproductive, respi...",Isolation and Sequence Analysis of Highly Path...


In [103]:
with pd.ExcelWriter(path+R'processed_data/community_content_analysis_50topwords.xlsx') as writer:  
    df_communities_topwords.to_excel(writer, sheet_name='word_analysis', index = False)
    community_analysis_topwords.to_excel(writer, sheet_name='publication_analysis', index = False)